# Persistence with DynamoDB

You have your langraph persistence setup as described in the [notebook](./persistence.ipynb). Now you want to use DynamoDB as your persistence layer. This notebook will guide you through the process of setting up DynamoDB as your persistence layer.

## Setup

First we need to install the packages required

In [1]:
!pip install --quiet -U langchain boto3


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Persistence

To add in persistence with DynamoDB, we pass in a checkpoint when compiling the graph

In [3]:
import boto3
from langchain_community.checkpoints.dynamodb import DynamoDbSaver

boto3_session = boto3.Session(region_name="us-west-2")

checkpointDynamoDbParams = {
    "table_name": "langgraph-checkpoints",
    "boto3_session": boto3_session,
}

memory = DynamoDbSaver.from_params(**checkpointDynamoDbParams)

In [6]:
from langgraph.graph import END, MessageGraph


# Define the function that calls the model
def call_model(messages):
    return "model output"


# Define the function to execute tools
def call_tool(messages):
    return "tool output"


# Define a new graph
workflow = MessageGraph()

# add FAKE nodes and edges to the graph...
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    False,
    {
        "continue": "action",
        "end": END,
    },
)
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory)

## Interacting with the agent

Ok, yes, this graph is not going to work, but this is just to show you how to setup the DynamoDB checkpoint!